In [1]:
from pprint import pprint
import json
import csv
import tweepy
import yaml
import time

from pathlib import Path
from time import sleep
import progressbar
import re

In [2]:
# Set up cache directories
tweet_cache_dir = Path('/Users/kallewesterling/_twitter_cache/tweets/')
user_cache_dir = Path('/Users/kallewesterling/_twitter_cache/users/')

if not tweet_cache_dir.is_dir(): tweet_cache_dir.mkdir(parents=True)
if not user_cache_dir.is_dir(): user_cache_dir.mkdir(parents=True)

In [3]:
# Set up tweepy
class _TwitterCredentials():

    def __init__(self):
        with open('../conference-documentation/credentials.yml') as f: self._ = yaml.load(f)

    def __getitem__(self, i):
        return self._[i]

twitter_credentials = _TwitterCredentials()

auth = tweepy.OAuthHandler(twitter_credentials['consumer_key'], twitter_credentials['consumer_secret'])
auth.set_access_token(twitter_credentials['access_token'], twitter_credentials['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [19]:
def expand_tags_tsv(file, _filter=False, filter_field=None, filter_value=None, done=[]):
    if Path(file).is_file(): _ = [Path(file)]
    elif Path(file).is_dir(): _ = Path(file).glob(pattern="*.tsv")
    else:
        raise RuntimeError(f"Cannot interpret passed argument: {file}.")
    for file in _:
        if file.name not in done:
            empty_rows, unable_to_interpret, saved_from_twitter, saved_from_tags = 0, 0, 0, 0

            with Path(file).open("r") as f:
                _len = len(f.readlines())
            with Path(file).open("r") as f:
                print(f"Reading {Path(file).name}...")
                time.sleep(1)
                bar = progressbar.ProgressBar(max_value=_len).start()
                reader = csv.DictReader(f, delimiter='\t')
                for i, rows in enumerate(reader):
                    stop = False

                    bar.update(i)
                    if not rows['created_at'] and not rows['from_user'] and not rows['text']:
                        pass
                    try:
                        id_int = int(rows['id_str'])
                        _dict = tags_to_dict(rows)
                        if _filter: 
                            try:
                                if filter_value in _dict[filter_field]: stop = False
                                else: stop = True
                            except TypeError:
                                stop = True

                        if not stop:
                            tweet_cache = tweet_cache_dir / _dict["id_str"]
                            if not tweet_cache.is_file():
                                ## First, check twitter for tweet here... 
                                try:
                                    live_tweet = api.get_status(_dict['id_str'], tweet_mode="extended")
                                    _json = live_tweet._json
                                    _json['json_source'] = 'Twitter'

                                    _json_user = _json['user']
                                    _json['user'] = _json_user['id']

                                    with Path(tweet_cache).open("w+") as f:
                                        json.dump(_json, f)

                                    user_cache = user_cache_dir / str(_json_user["id"])
                                    if not user_cache.is_file():
                                        with Path(user_cache).open("w+") as f:
                                            _json_user['json_source'] = 'Twitter'
                                            json.dump(_json_user, f)

                                    saved_from_twitter += 1

                                except tweepy.TweepError as e:
                                    ## No tweet available:
                                    with Path(tweet_cache).open("w+") as f:
                                        _dict['error'] = str(e)
                                        json.dump(_dict, f)
                                    saved_from_tags += 1

                                    try:
                                        user_cache = user_cache_dir / str(_dict["user"]["id_str"])
                                        if not user_cache.is_file():
                                            live_user = api.get_user(_dict["user"]["id_str"])
                                            _json_user = live_user._json

                                            # remove the user's latest status
                                            try: del _json_user['status']
                                            except KeyError: pass

                                            _json_user['json_source'] = 'Twitter'

                                            with Path(user_cache).open("w+") as f:
                                                json.dump(_json_user, f)

                                    except tweepy.TweepError as e:
                                        if not user_cache.is_file():
                                            _json_user = {"error": str(e)}
                                            with Path(user_cache).open("w+") as f:
                                                json.dump(_json_user, f)


                    except ValueError:
                        if rows['id_str'] == "":
                            empty_rows += 1
                        else:
                            try:
                                # print(f"Warning: Could not interpret row {i} in file {Path(file).name} due to unintelligibility but trying to find tweet ID...")
                                _ = re.search(pattern="\/statuses\/([0-9]+)'\)", string=str(rows))
                                g = _.groups()
                                # print(f"Good news! Found ID: {g}")
                                tweet_cache = tweet_cache_dir / g[0]
                                if not tweet_cache.is_file():
                                    ## First, check twitter for tweet here... 
                                    try:
                                        if _filter: 
                                            try:
                                                if filter_value in _dict[filter_field]: stop = False
                                                else: stop = True
                                            except TypeError:
                                                stop = True

                                        if not stop:
                                            live_tweet = api.get_status(g[0], tweet_mode="extended")
                                            _json = live_tweet._json
                                            _json['json_source'] = 'Twitter'

                                            _json_user = _json['user']
                                            _json['user'] = _json_user['id']

                                            with Path(tweet_cache).open("w+") as f:
                                                json.dump(_json, f)

                                            user_cache = user_cache_dir / str(_json_user["id"])
                                            if not user_cache.is_file():
                                                with Path(user_cache).open("w+") as f:
                                                    _json_user['json_source'] = 'Twitter'
                                                    json.dump(_json_user, f)
                                    except tweepy.TweepError as e:
                                        ## No tweet available:
                                        with Path(tweet_cache).open("w+") as f:
                                            _dict['error'] = str(e)
                                            json.dump(_dict, f)
                                    except:
                                        print(f"An error occurred in file {f}")
                            except:
                                unable_to_interpret += 1
                bar.finish()
                print(f"---- done processing {Path(file).name} - report: -----")
                print(f"- {empty_rows} empty rows encountered.")
                print(f"- Unable to interpret {unable_to_interpret} rows.")
                print("\n")
                print(f"- {saved_from_twitter} tweets saved from Twitter.")
                print(f"- {saved_from_tags} tweets saved from TAGS.")
                print("\n")

def tags_to_dict(rows):
    if not rows['id_str']:
        print("STOP")
        pprint(rows)
        exit()
    else:
        _dict = {
            'created_at': rows['created_at'],
            'id': int(rows['id_str']),
            'id_str': rows['id_str'],
                            'from_user': rows['from_user'],
            'full_text': rows['text'],
            'geo_coordinates': rows['geo_coordinates'],
            'lang': rows['user_lang'],
            'in_reply_to_user_id_str': rows['in_reply_to_user_id_str'],
            'in_reply_to_screen_name': rows['in_reply_to_screen_name'],
            'user': {
                'id_str': rows['from_user_id_str'],
                'followers_count': rows['user_followers_count'],
                'friends_count': rows['user_friends_count'],
            },
            'in_reply_to_status_id_str': rows['in_reply_to_status_id_str'],
            'source': rows['source'],
            'profile_image_url': rows['profile_image_url'],
            'entities_str': rows['entities_str'],
            'json_source': "TAGS"
        }
    return(_dict)

In [20]:
# The expand_tags_tsv function accepts whole directories or, if you prefer, individual TAGS archive sheets, saved as .tsv files
'''
expand_tags_tsv('../../datasets/tags-tsv/burlesque/TAGS - burlesque 1 - Archive.tsv', _filter=True, filter_field="full_text", filter_value="burlesque") # done
'''

'\nexpand_tags_tsv(\'../../datasets/tags-tsv/burlesque/TAGS - burlesque 1 - Archive.tsv\', _filter=True, filter_field="full_text", filter_value="burlesque") # done\n'

In [21]:
'''
expand_tags_tsv('../../datasets/tags-tsv/burlesque/TAGS - burlesque 10 - Archive.tsv', _filter=True, filter_field="full_text", filter_value="burlesque")
'''

'\nexpand_tags_tsv(\'../../datasets/tags-tsv/burlesque/TAGS - burlesque 10 - Archive.tsv\', _filter=True, filter_field="full_text", filter_value="burlesque")\n'

In [25]:
expand_tags_tsv('../../datasets/tags-tsv/burlesque/', _filter=True, filter_field="full_text", filter_value="burlesque", done=['TAGS - burlesque 60 - Archive.tsv'])

Reading TAGS - burlesque 49 - Archive.tsv...


100% (44922 of 44922) |##################| Elapsed Time: 0:00:04 Time:  0:00:04


---- done processing TAGS - burlesque 49 - Archive.tsv - report: -----
- 0 empty rows encountered.
- Unable to interpret 0 rows.


- 0 tweets saved from Twitter.
- 0 tweets saved from TAGS.


Reading TAGS - burlesque 17 - Archive.tsv...


100% (47675 of 47675) |##################| Elapsed Time: 0:00:05 Time:  0:00:05


---- done processing TAGS - burlesque 17 - Archive.tsv - report: -----
- 0 empty rows encountered.
- Unable to interpret 0 rows.


- 0 tweets saved from Twitter.
- 0 tweets saved from TAGS.


Reading TAGS - burlesque 39 - Archive.tsv...


...

---- done processing TAGS - burlesque 48 - Archive.tsv - report: -----
- 40 empty rows encountered.
- Unable to interpret 71 rows.


- 0 tweets saved from Twitter.
- 0 tweets saved from TAGS.


Reading TAGS - burlesque 12 - Archive.tsv...


100% (117636 of 117636) |################| Elapsed Time: 0:00:11 Time:  0:00:11


---- done processing TAGS - burlesque 12 - Archive.tsv - report: -----
- 1281 empty rows encountered.
- Unable to interpret 35 rows.


- 0 tweets saved from Twitter.
- 0 tweets saved from TAGS.


Reading TAGS - burlesque 16 - Archive.tsv...


100% (47044 of 47044) |##################| Elapsed Time: 0:00:04 Time:  0:00:04


---- done processing TAGS - burlesque 16 - Archive.tsv - report: -----
- 14 empty rows encountered.
- Unable to interpret 2 rows.


- 0 tweets saved from Twitter.
- 0 tweets saved from TAGS.




In [ ]:
expand_tags_tsv('../../datasets/tags-tsv/male striptease/')
expand_tags_tsv('../../datasets/tags-tsv/boy-lesque/')
expand_tags_tsv('../../datasets/tags-tsv/male burlesque/')
expand_tags_tsv('../../datasets/tags-tsv/boylesque/')
expand_tags_tsv('../../datasets/tags-tsv/burlesk OR burleycue OR burly-q/')